# Tutorial from official docs

In [19]:
import os
import json
ROOT = os.getcwd()
import pickle

import getpass

In [12]:
os.environ["LANGCHAIN_TRACING_V2"] = "True"
key_path = os.path.join(ROOT, 'secrets.json')

with open(key_path, 'r') as file:
    key = json.load(file)

os.environ["LANGCHAIN_API_KEY"] = key["LANGCHAIN_API_KEY"] 
os.environ["OPENAI_API_KEY"] = key["OPENAI_API_KEY"]

# Import LangChain

In [42]:
import bs4

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub

# Load

In [4]:
strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths = ('https://lilianweng.github.io/posts/2023-06-23-agent/',), #each link will parsed into 1 langchain Document
    bs_kwargs={"parse_only": strainer}
)

docs = loader.load() #docs is a list of langchain_core.documents.base.Document 
print('doc len = ', len(docs[0].page_content))
print('doc type = ', type(docs[0]))
print('content = ')
print(docs[0].page_content[:200])

doc len =  43131
doc type =  <class 'langchain_core.documents.base.Document'>
content = 


      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a 


# Split

- text too long
- can't fit model
- hard to search

Split Document into chunk of 1000 chars, 200 overlapping chars (retain important context)

use RecursiveCharacterTextSplitter

In [16]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)

all_splits = splitter.split_documents(docs)

print('all splits = ', len(all_splits))
print('each split page_content = ', len(all_splits[0].page_content))
print('each split meta_data = ', all_splits[0].metadata)

all splits =  66
each split page_content =  969
each split meta_data =  {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 8}


# Index (Store)

- embed content of each split
- store in vector database
- take search query --> embed search query -> perform similarity search -> identify most similiar embeddings to query embedding, (simplest is cosine similarity)
- embed and store all document splits using Chroma vector and OpenAIEmbeddings


In [17]:
vector_store = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings()) #OpenAI requires payment to use embedding


# Retrieve

In [18]:
#Calling this cost some OpenAI credits
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke('what are the approaches related to task decomposition')


[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 2192}, page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'), Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 1585}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task us

In [23]:
with open("retrieved_docs.pickle", 'wb') as file:
    pickle.dump(retrieved_docs, file)

In [31]:
print(retrieved_docs)
print(type(retrieved_docs))
print(len(retrieved_docs))
print(retrieved_docs[0])

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 2192}, page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'), Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 1585}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task us

# Generate

- load gpt model 
- use rag prompt template, document: https://smith.langchain.com/hub/rlm/rag-prompt?organizationId=abf9a848-7efe-566a-b2cb-31b1b8db5108


In [36]:
llm = ChatOpenAI(model="gpt-4o-mini")
prompt = hub.pull("rlm/rag-prompt") #

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filter question \nContext: filter context \nAnswer:")]


In [41]:
example_messages = prompt.invoke({
    'context': 'filter context',
    'question': 'filter question'
}).to_messages()

print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filter question 
Context: filter context 
Answer:


# Use LangChain Expression Language (LCEL) to define chain

- pipe together components
- automatically trace chain in LangSmith
- stream, async, batched out of the box


each object is cast into a Runnable.

In [43]:
def format_docs(docs: list):
    return "\n\n".join((doc.page_content for doc in docs))

In [47]:
#Runnable interface will take the | operator as a 
# chaining logic (unlike normal or operation)
# take the output of previous statement and feed to current statement as input
# --> create a chain
# https://www.pinecone.io/learn/series/langchain/langchain-expression-language/

rag_chain = (
    #format_docs is cast into RunnableLambda
    #context and question is cast to RunnableParallel
    #pass question through retriever -> list of Document --> pass through format_docs --> string
    # RunnablePassThrough passes through input unchanged
    {'context': retriever | format_docs, "question": RunnablePassthrough()}
    | prompt #build formatted prompt, ready for inference
    | llm #pass prompt through LLM
    | StrOutputParser() #parse the LLM output into string
)

for chunk in rag_chain.stream("What is task decomposition"):
    print(chunk , end = '', flush = True)


Task decomposition is the process of breaking down a complicated task into smaller, manageable steps. Techniques like Chain of Thought (CoT) and Tree of Thoughts help models think step by step, allowing for better performance on complex tasks. This can be achieved through simple prompting, task-specific instructions, or human inputs.

# Using built-in chains

In [49]:
# add retrieval step and propagate retrieved context through the chain
# stuff content into prompt, raw retrieved context, produce answer 
# using retrieved context and query
from langchain.chains import create_retrieval_chain 

from langchain.chains.combine_documents import create_stuff_documents_chain 
from langchain_core.prompts import ChatPromptTemplate



In [55]:
# This is a way to declare multiline string in Python
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

print(system_prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        ('system', system_prompt),
        ('human', '{input}'),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt) 
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({'input': 'what is task decomposition'})

print(response['answer'])

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.

{context}
Task decomposition is the process of breaking down a complex task into smaller, more manageable subtasks. This approach, often facilitated by techniques like Chain of Thought (CoT), helps improve performance by allowing a model to address each subtask step by step. Decomposition can be achieved through prompts, specific instructions, or human input.


In [56]:
# context is propagated retrieved document through into output
for doc in response['context']:
    print(doc)
    print()

page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 2192}

page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; 